### Decorator Application: Decorating Classes

> Aaron's Experiments on Decorator Application - Decorating Classes

## <font color=palevioletred> Aaron Title
### **Python 裝飾器進階：Monkey Patching 與自動補齊比較運算符**  

#### **Abstract**  
Python 裝飾器（decorators）不僅能用來修飾函式，還能用來修改類別的行為，例如自動補齊比較運算符。Monkey patching 則是一種在執行時修改類別或函式的方法，雖然靈活但可能影響程式的可維護性。透過 `functools.total_ordering`，我們可以讓 Python 自動推導 `<=`、`>=` 和 `>`，只需手動定義 `<` 和 `==`，大幅減少程式碼重複。這篇文章將深入探討這些技巧，並提供實際應用案例來提升 Python 編程的效率與可讀性。

### **Jupyter Notebook 總結：裝飾器應用（裝飾類別）**

這份筆記探討如何將 **裝飾器（decorator）** 應用於 **類別（class）**，而不僅限於函式（function）。主要涵蓋的內容如下：

---

### **1. 類別裝飾器的概念**
- 以往我們使用裝飾器來裝飾函式與方法。
- 但裝飾器也可以作用於類別，使其可以修改類別的屬性或方法，然後回傳修改後的類別。

---

### **2. Monkey Patching（猴子補丁）**
- **Monkey patching** 指的是在 **執行時動態修改或擴展程式碼**。
- 在 Python 中，我們可以在運行時修改大多數用 Python 撰寫的類別（但內建類別如 `str`、`list` 等不行）。

#### **範例：為 `Fraction` 類別新增方法**
```python
from fractions import Fraction
Fraction.speak = lambda self: "這是一隻遲到的鸚鵡。"
```
- 這段程式碼為 `Fraction` 類別新增一個 `speak()` 方法，即使 `Fraction` 是標準函式庫的一部分。

#### **更有用的範例：新增判斷是否為整數的方法**
```python
Fraction.is_integral = lambda self: self.denominator == 1
```
- 這個方法判斷該 `Fraction` 是否為整數（即分母是否為 `1`）。

---

### **3. 使用函式修改類別**
- 與其手動修改類別，不如撰寫一個函式來完成這件事，並將其作為裝飾器使用。

#### **範例：建立一個裝飾器來新增方法**
```python
def dec_speak(cls):
    cls.speak = lambda self: "這是一隻非常遲到的鸚鵡。"
    return cls

Fraction = dec_speak(Fraction)
```
- `dec_speak` 這個函式接收一個類別，並動態為其新增 `speak` 方法。

#### **使用 `@` 語法裝飾類別**
```python
@dec_speak
class Parrot:
    def __init__(self):
        self.state = "late"
```
- 這樣一來，`Parrot` 類別的所有物件都會擁有 `speak()` 方法。

---

### **4. 更實用的類別裝飾器**
- 例如，我們可以建立一個裝飾器來為類別新增 **除錯資訊（debugging info）** 方法。

#### **除錯裝飾器**
```python
from datetime import datetime, timezone

def debug_info(cls):
    def info(self):
        results = []
        results.append(f"時間: {datetime.now(timezone.utc)}")
        results.append(f"類別: {self.__class__.__name__}")
        results.append(f"記憶體位址: {hex(id(self))}")

        if vars(self):
            results.extend(f"{k}: {v}" for k, v in vars(self).items())

        return results
    
    cls.debug = info
    return cls
```
- 這個裝飾器會為類別新增一個 `debug()` 方法，當我們呼叫 `debug()` 時，它會回傳該物件的 **記憶體位址、類別名稱、變數值** 等資訊。

#### **將 `debug_info` 應用於類別**
```python
@debug_info
class Person:
    def __init__(self, name, birth_year):
        self.name = name
        self.birth_year = birth_year
```
- 現在 `Person` 物件將擁有 `debug()` 方法，可用於檢視物件的相關資訊。

---

### **重點整理**
1. **Monkey patching（猴子補丁）** 允許我們在運行時修改類別，但應謹慎使用。
2. **類別裝飾器（Class Decorators）** 可以修改類別並提供額外的功能。
3. **裝飾器應用場景**：
   - 可用來擴展類別的功能，例如 **自動加入除錯工具、日誌記錄、驗證機制等**。
4. **`@decorator` 語法** 適用於類別，就像它適用於函式一樣。

這份筆記示範了一些 **實用的類別裝飾器**，這些技巧可以用於 **記錄日誌、除錯、擴展第三方類別**，甚至為框架提供可重用的功能！🚀

到目前為止，我們已經學習了如何裝飾函式。這表示我們可以裝飾使用 `def` 語句定義的函式（可以使用 `@` 語法，或是較長的寫法）。由於類別方法（class methods）也是函式，因此它們也可以被裝飾。Lambda 表達式同樣可以被裝飾（需使用較長的寫法）。

但如果你仔細思考我們的裝飾器（decorator）是如何運作的，它們接受一個函式作為參數，並返回另一個函式——通常是一個使用原始函式的閉包（closure）。

我們可以使用相同的概念，不是接收一個函式，而是接收一個類別（class）。我們可以在裝飾器內部引用該類別，修改它，然後返回修改後的類別。

---

首先，我們來看看所謂的 **monkey patching**（猴子補丁）。本質上，它就是在 **執行時** 修改或擴展我們的程式碼。

例如，我們可以在執行時修改或新增類別的屬性，甚至是模組（module）的屬性。

在 Python 中，許多我們使用的類別都可以在執行時修改（但內建類別，如字串 `str`、列表 `list` 等，則不能）。

然而，在 Python 中撰寫的類別，例如我們自己寫的類別，甚至是 Python 標準函式庫中的類別（只要它們是用 Python 而不是 C 編寫的），都是可以修改的。例如，`fractions` 模組中的 `Fraction` 類別就可以進行 monkey patching。

不過，儘管我們可以這樣做，但這並不代表我們應該這樣做！Monkey patching 可能非常有用，但不應該只是因為「可以」而去做——就像我們稍後將看到的，應該有充分的理由才使用它。

此外，通常來說，對特殊方法（`__???__`，如 `__len__`）進行 monkey patching 是個壞主意，因為 Python 查找這些方法的方式可能會導致這樣的修改無效。

---

### 第二部分

是的，這顯然是個無厘頭的例子，但你應該能理解，即使我們沒有對類別的直接控制權，或者類別已經定義後，我們仍然可以向其新增屬性。

如果你想要一個更有用的方法，那麼我們可以新增一個方法來檢查 `Fraction` 是否為整數（即分母為 `1`）。

利用這種技術，我們可以為 `Fraction` 類別新增一個 *reciprocal*（倒數）屬性。然而，由於這可能只是一個一次性的需求（畢竟，我們能有多少 `Fraction` 類別需要額外新增一個 `reciprocal` 呢？），因此不需要使用裝飾器。裝飾器通常用於能夠在更一般的情境下重複使用的功能。

這些例子可能都相當簡單，甚至顯得無關緊要。

那麼，為什麼要提到這個技術呢？

因為這種技術可以應用在更有趣的情境中。

舉個例子，假設我們在除錯（debug）時，通常會想要檢查物件的各種屬性，例如記憶體地址（memory address）、目前的狀態（屬性值），以及生成除錯資訊的時間。

---

### 運算符的比較

嗯，我們可能會期望 `p1` 和 `p2` 是相等的，因為它們擁有相同的座標。但 Python 預設會比較物件的記憶體地址，而不是值，因為我們的類別並未實作 `__eq__` 方法（該方法用於 `==` 比較）。

對於 `<` 運算符，我們需要在類別中實作 `__lt__` 方法，而對於 `==`，我們需要實作 `__eq__` 方法。

其他比較運算符則需要實作對應的方法，例如：
- `__le__` (`<=`)
- `__gt__` (`>`)
- `__ge__` (`>=`)

我們現在要為 `Point` 類別新增 `__lt__` 和 `__eq__` 方法。

我們將定義 `Point` 物件的比較規則：離原點（即大小為絕對值）較近的 `Point` 物件被視為「較小」。

---

### Python 自動處理部分運算符

哇，既然我們已經實作了 `<` 和 `==`，那麼 Python 會自動幫我們實作 `>`（即「不是 `<` 且不是 `==`」）嗎？

不完全是！實際上，當我們執行 `p1 > p4` 時，Python 會自動將它轉換為 `p4 < p1`，並使用我們已經定義的 `__lt__` 方法來進行比較。

但 Python 並不會自動實作其他運算符，例如 `>=` 和 `<=`：

雖然我們可以類似地為 `>=`、`<=` 和 `>` 實作方法，但觀察一下這些運算符的關係：
- `a <= b` 若且唯若 `a < b 或 a == b`
- `a > b` 若且唯若 `not(a < b) 且 a != b`
- `a >= b` 若且唯若 `not(a < b)`

因此，為了更通用化，我們可以建立一個裝飾器，使其能夠自動實作這三個運算符，只要 `==` 和 `<` 已經定義好即可。這樣，我們可以對 **任何** 僅實作了這兩個運算符的類別進行裝飾，使其擁有完整的比較功能。

---

### 這樣的實作方式是否完美？

事實上，上述方法並 **不是** 良好的實作方式。我們並未檢查物件類型是否相容，且沒有在適當情況下回傳 `NotImplemented`。此外，我們直接使用了 `<` 和 `==` 運算符，而不是 `__lt__` 和 `__eq__` 方法。這麼做的確簡單，但我們稍後會改進這種方式。

你可能會問，為什麼我要直接使用 `__lt__` 而不是 `<` 運算符？這是因為我希望能夠檢查操作結果，而不會在運算符未實作時拋出例外。如果我們的 `total_ordering` 裝飾器沒有正確處理 `self < other`，Python 會嘗試反向評估 `other > self`，如果這個評估也失敗，Python 可能會進一步嘗試反向運算，這將導致無限遞迴（最終導致堆疊溢位）。這其實曾經是 Python 標準函式庫中 `total_ordering` 裝飾器的一個 bug，並在 Python 3.4 中修復。

---

### `functools.total_ordering`

通常來說，當 `==` 運算符與 **任意一個** 其他比較運算符（`<`、`<=`、`>`、`>=`）已經定義時，其他運算符都可以被推導出來。

我們的裝飾器目前要求 `==` 和 `<` 兩者都被實作。但我們可以進一步改進它，使其只需 `==` 和 **任意一個** 其他運算符，這樣就更加通用。然而，這樣的裝飾器會變得更為複雜。

實際上，Python 已經內建了這樣的功能——你猜對了，它就在 `functools` 模組裡！

它的名稱是 `total_ordering`，讓我們來看看它的實際應用。


## <font color=palevioletred>說人話～

簡單來說，這段內容主要在講 **Python 裝飾器（decorators）** 和 **monkey patching（猴子補丁）**，還有 **如何讓類別支援比較運算符**。我們來拆開來解釋：

### 1. **裝飾器不只可以用來裝飾函式，還可以用來裝飾類別**
   - 裝飾器最常見的用法是用 `@` 來裝飾函式（像 `@staticmethod`）。
   - 但其實類別（class）也可以被裝飾。
   - 這意味著我們可以寫一個裝飾器來修改類別的行為，而不需要直接改動類別的程式碼。

---

### 2. **什麼是 Monkey Patching（猴子補丁）？**
   - **Monkey patching** 指的是 **在程式執行時動態修改類別或函式**。
   - 例如，原本 `Fraction` 類別沒有 `is_integer()` 方法，我們可以 **在執行時** 幫它加上一個：
     ```python
     from fractions import Fraction

     def is_integer(self):
         return self.denominator == 1

     Fraction.is_integer = is_integer  # 給 Fraction 類別動態加上新方法
     ```
   - 這種技巧可以修改現有的類別或函式，但 **濫用可能導致程式變得難以維護**。

---

### 3. **Python 的類別比較運算**
   - 假設我們有一個 `Point` 類別，代表平面上的點：
     ```python
     class Point:
         def __init__(self, x, y):
             self.x = x
             self.y = y
     ```
   - 如果我們用 `p1 == p2` 來比較兩個點，Python 只會比較「記憶體地址」，而不是 `x`、`y` 值。
   - 要讓 `Point` 能用 `==` 來比較，我們需要實作 `__eq__` 方法：
     ```python
     def __eq__(self, other):
         return self.x == other.x and self.y == other.y
     ```

   - 如果我們想支援 `<`（小於）運算符呢？我們可以定義：
     ```python
     def __lt__(self, other):
         return (self.x**2 + self.y**2) < (other.x**2 + other.y**2)
     ```
     這樣 `p1 < p2` 會根據距離原點的遠近來判斷。

   - 但 Python **不會自動補齊其他比較運算符（如 `>=`、`>`、`<=`）**，所以如果我們只定義了 `<` 和 `==`，`>` 可能會自動反轉為 `<`，但 `<=` 和 `>=` 就不會自動存在。

---

### 4. **用裝飾器自動補齊比較運算符**
   - 既然定義 `<` 和 `==` 就能推出 `<=`、`>`、`>=`，我們可以寫一個裝飾器來自動補齊這些運算符：
     ```python
     from functools import total_ordering

     @total_ordering
     class Point:
         def __init__(self, x, y):
             self.x = x
             self.y = y

         def __eq__(self, other):
             return self.x == other.x and self.y == other.y

         def __lt__(self, other):
             return (self.x**2 + self.y**2) < (other.x**2 + other.y**2)
     ```
   - 這樣 `Point` 只要定義 `==` 和 `<`，Python 就會幫我們補齊 `<=`、`>`、`>=`！

---

### **結論**
1. Python 裝飾器不只可以裝飾函式，還可以裝飾類別來改變它的行為。
2. Monkey patching 可以動態修改類別，但不要濫用。
3. 如果類別需要支援比較運算符（`<`、`>`、`==`），可以用 `functools.total_ordering` 來讓 Python 自動補齊剩下的比較運算符，省去重複寫 `>=`、`<=`、`>` 的麻煩。

這樣講比較白話吧？😆

### Decorator Application: Decorating Classes

But if we think about how our decorators work, they take a single parameter, a function, and return some other function - usually a closure that uses the original functio that was passed as an argument.

We have so far worked with decorating functions. This means we can decorate functions defined with a `def` statement (we can use the `@` syntax, or the long form). Since class methods are functions, they can be decorated too. Lambda expresson can also be decorated (using the long form).

We could use the same concept to accept, not a function, but a class instead.

First, we look at something called monkey patching. It boils down to modifying or extending our code at run time.

For example, we can modify or add attributes to classes at run time. Modules too.

In Pythonm many of the classes we use can be modified at run time (built-ins like strings, lists, and so on, cannot)

But classes written in Python, such as the ones we write, and even library classes, as long as they are written in Python,m not C, can. For example `Fraction` in the `fractions` module can be monkey patch.

Just because we can do something, does not mean we should! Monkey patching can be extremely useful, but don't do it just because you can - as always there should be a real reason to do it, as we'll see in a bit. -- Monkey Patching could be extremely useful, but don't do it just because you can - as always there should be a real reason to do it, as we'll see in a bit.

Also, in general it is a bad idea to monkey patch the special methods __???__ (such as __len__) as this will often not work due to how these methods are searched for Python



但是如果我們思考裝飾器的運作方式，它們接受一個參數，也就是一個函式，並回傳另一個函式——通常是閉包，並且使用原始傳入的函式。

到目前為止，我們一直在處理裝飾函式。這意味著我們可以裝飾用 `def` 語句定義的函式（我們可以使用 `@` 語法，或是長格式）。由於類別方法也是函式，它們也可以被裝飾。Lambda 表達式也可以被裝飾（使用長格式）。

我們可以使用相同的概念來接受一個類別，而不是一個函式。

首先，我們來看看所謂的「猴子補丁」(monkey patching)。它基本上是指在執行時修改或擴展我們的程式碼。
舉例來說，我們可以在執行時修改或新增類別的屬性，模組也是如此。



在 Python 中，我們使用的許多類別可以在執行時被修改（內建類別像是字串、列表等等不能被修改）。

但用 Python 撰寫的類別，例如我們自己寫的類別，甚至是庫中的類別，只要它們是用 Python 撰寫的（不是 C 語言），就可以被修改。例如 fractions 模組中的 Fraction 類別可以被猴子補丁。

但是，僅僅因為我們能夠做到，並不代表我們應該這樣做！猴子補丁可以非常有用，但不要僅僅因為你可以就去做——一如既往，應該有一個真正的理由來這麼做，稍後我們會深入探討這點。——猴子補丁可以非常有用，但不要僅僅因為你可以就去做——一如既往，應該有一個真正的理由來這麼做，稍後我們會深入探討這點。

此外，一般來說，猴子補丁特殊方法 __???__（例如 __len__）是個壞主意，因為這些方法通常會因為 Python 搜尋這些方法的方式而無法正常運作。



## Aaron註解：

### <font color=palevioletred> 這段文字其實在討論兩個概念：裝飾器和猴子補丁。

### 總結
#### `裝飾器`讓你可以在不直接修改函式的情況下，對它進行包裝或擴展功能。
#### `猴子補丁`是直接修改現有的程式碼或類別，雖然有時候很有用，但需要小心使用，避免造成不必要的問題。


### <font color=lightseagreen> 裝飾器  
#### 裝飾器（decorator）基本上是用來改變函式行為的工具。你可以把一個函式「包裝」成另一個函式，通常用來添加額外的功能或修改原本的行為。裝飾器一般會接受一個函式，然後返回一個修改過的函式。這也包括了像是裝飾類別方法、Lambda 表達式等。

#### 簡單來說，裝飾器是讓你在不改動原始函式程式碼的情況下，對它進行一些額外的處理。


### <font color=dodgerblue>
### 猴子補丁（Monkey Patching）  
#### 猴子補丁指的是在程式運行時修改或擴展現有的類別或模組。這樣可以改變某些行為或為它們新增功能。Python 允許我們在運行時對一些類別進行這種修改，尤其是對 Python 本身寫的類別（例如我們自己寫的類別或第三方的庫），但對像是字串、列表等內建的類別不能做修改。

#### 然而，這並不意味著你應該隨便使用猴子補丁。它雖然有時候非常有用，但如果沒有真正的需要，隨便修改別人的程式碼可能會帶來不必要的麻煩。而且，對於 Python 的特殊方法（像是 __len__），進行猴子補丁通常會遇到問題，因為 Python 會特別尋找這些方法，可能會破壞某些行為。


### <font color=olive> Aaron 補充：

#### 為什麼叫「猴子補丁」？
##### 這個名稱來自於「猴子」這個動物的特性。猴子是靈活且擅長改變和適應的，通常會隨意地抓取周圍的東西並加以改造。這個比喻形容的是在程序運行中，對現有的代碼進行隨意修改或即時改動，就像猴子亂抓東西一樣。

##### 換句話說，「猴子補丁」*`是指對現有代碼進行隨機且直接的修改，而不是通過有計劃的結構化方式進行改變`*。這種做法可能很靈活，但也容易引發一些意料之外的問題或錯誤，就像猴子在亂抓東西時，可能會破壞某些東西一樣。

#### 猴子補丁的特點
> - 靈活性：就像猴子可以隨意地抓取和改變物品，猴子補丁讓你可以在程式運行時改變現有的類別或方法，動態修改行為。
> - 危險性：如果不小心使用，可能會破壞原來的設計或產生不可預測的問題，就像猴子亂抓東西可能會損壞東西一樣。

##### 所以，「猴子補丁」這個名字就形象地反映了這種既靈活又充滿風險的做法。它是一種快速的解決方案，但往往沒有經過深思熟慮，並且需要謹慎使用。

##### 總結來說，就是猴子因為會隨意改變東西，所以用來形容這種「隨便」修改程式碼的做法，也帶有一點警告意味在裡面！

In [1]:
from fractions import Fraction

In [2]:
Fraction.speak = lambda self: 'This is a late parrot'

In [3]:
f = Fraction(2, 3)

In [4]:
f

Fraction(2, 3)

In [5]:
f.speak()

'This is a late parrot'

沒錯，這看起來很無聊，但我們應該明白這個概念：即使你無法直接控制一個類別，或者當你的類別已經定義後，你仍然可以動態地為它添加屬性。

如果你想要一個更實用的方法，怎麼樣的話我們來實現一個可以告訴我們某個 Fraction 是否為整數的方法呢？（也就是分母為 1）。

In [6]:
Fraction.is_integral = lambda self: self.denominator == 1

In [7]:
f1 = Fraction(1, 2)
f2 = Fraction(10, 5)

In [8]:
f1.is_integral()

False

In [9]:
f2.is_integral()

True

##### Now, we can make this change to the class by calling a function to do it instead:

In [10]:
def dec_speak(cls):
    cls.speak = lambda self: 'This is a very late parrot.'
    return cls

In [11]:
Fraction = dec_speak(Fraction)

##### 上面這段其實就是裝飾器啦

In [12]:
f = Fraction(10, 2)

In [13]:
f.speak()

'This is a very late parrot.'

##### 上面用函數來修飾我們的class，我們也可以用下面的 @ 語法

In [15]:
@dec_speak
class Parrot:
    def __init__(self):
        self.state = 'late'

##### 使用這種技術，我們可以舉個例子，為 Fraction 類別添加一個有用的 reciprocal 屬性（倒數屬性）。不過，當然了，這樣的改動可能只是一次性的需求（畢竟，多少個 Fraction 類別是你會希望在之後添加倒數屬性的呢？），所以其實沒有必要使用裝飾器。裝飾器的優勢在於它們能夠被重複使用，並且在更通用的情境下發揮作用。

In [20]:
Fraction.recip = lambda self: Fraction(self.denominator, self.numerator)

In [21]:
f = Fraction(2, 3)

In [22]:
f

Fraction(2, 3)

In [24]:
f.recip()

Fraction(3, 2)

這些例子相當簡單，且並不太實用。

那麼，為什麼要提這個呢？

因為這個技術其實可以用來做一些更有趣的事情。

#### <font color=olive> 作為第一個例子，假設你通常會`檢查物件的各種屬性`來進行`除錯`，可能會檢查它的
> - 記憶體位址、
> - 當前狀態（屬性值），
> - 以及生成除錯資訊的時間。

In [25]:
from datetime import datetime, timezone

In [29]:
def debug_info(cls):
    def info(self):
        results = []
        results.append(f'time: {datetime.now(timezone.utc)}')
        results.append(f'class: {self.__class__.__name__}')  # class name (try later: cls.__name__)
        results.append(f'id: {hex(id(self))}')

        if vars(self):
            results.append('vars:')
            for k, v in vars(self).items():
                results.append(f'{k}: {v}')
                
        # if using lists, the extend method and generators, and a more Pythonic way:
        # if vars(self):
        #    results.extend([f'{k}: {v}' for k, v in vars(self).items()])
        
        # return '\n'.join(results)
        return results
    
    cls.debug = info

    return cls
                
        

In [31]:
@debug_info
class Person:
    def __init__(self, name, birth_year):
        self.name = name 
        self.birth_year = birth_year
    
    def say_hi():
        return 'Hello there!~'

####     為什麼 debug_info 裝飾器會將 debug 方法加到 cls 類別中？
當你使用 @debug_info 裝飾器時，Python 會將這個裝飾器應用到類別 Person 上。裝飾器的作用是修改這個類別或增加一些行為。

在你的 debug_info 裝飾器中，debug 方法被添加到 cls（即 Person 類別）上。具體來說，這段代碼：

```python
cls.debug = info
```
將 debug 方法指向 info 函數。這裡的 info 函數就是`裝飾器內部定義的那個函數` (類似 inner)，它會返回一些有關物件的資訊（例如當前時間、類別名、物件 ID、屬性等）。因此，debug 方法成為了 Person 類別的一部分。

In [32]:
p1 = Person('John',  1939)

In [33]:
p1.debug()

['time: 2025-02-24 04:19:43.894657+00:00',
 'class: Person',
 'id: 0x112ebe390',
 'vars:',
 'name: John',
 'birth_year: 1939']

#### 當然我們也可以用來裝飾其他的classes，不是只有這個class

In [35]:
@debug_info
class Automobile:
    def __init__(self, make, model, year, top_speed_mph):
        self.make = make
        self.model = model
        self.year = year
        self.top_speed_mph = top_speed_mph
        self.current_speed = 0
        
    @property
    def speed(self):
        return self.current_speed

    @speed.setter
    def speed(self, new_speed):
        self.current_speed = new_speed

In [36]:
s = Automobile('Ford', 'Model T', 1908, 45)

In [37]:
s.debug()

['time: 2025-02-24 08:52:53.283446+00:00',
 'class: Automobile',
 'id: 0x112e9fed0',
 'vars:',
 'make: Ford',
 'model: Model T',
 'year: 1908',
 'top_speed_mph: 45',
 'current_speed: 0']

In [38]:
s.debug()

['time: 2025-02-24 08:53:08.101449+00:00',
 'class: Automobile',
 'id: 0x112e9fed0',
 'vars:',
 'make: Ford',
 'model: Model T',
 'year: 1908',
 'top_speed_mph: 45',
 'current_speed: 0']

In [39]:
s.speed = 20

In [40]:
s.debug()

['time: 2025-02-24 08:53:21.562855+00:00',
 'class: Automobile',
 'id: 0x112e9fed0',
 'vars:',
 'make: Ford',
 'model: Model T',
 'year: 1908',
 'top_speed_mph: 45',
 'current_speed: 20']

#### 下列的例子，我們裝飾整個 class，這可能會很有用

In [41]:
from math import sqrt

In [43]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __abs__(self):
        return sqrt(self.x**2 + self.y**2)
    
    def __repr__(self):
        return f'Point({self.x}, {self.y})'

In [44]:
p1, p2, p3 = Point(2, 3), Point(2, 3), Point(0, 0)

In [45]:
abs(p1)

3.605551275463989

In [46]:
p1, p2

(Point(2, 3), Point(2, 3))

In [48]:
p1 == p2

False

#### 哈哈， 我們可能原來想的是 `p1` ==  `p2`，因為他們的座標一樣。但是 Python 的預設是比較兩個 memory 的 addresses，因為我們沒有去重建 `__eq__` 方法，來客製化我們的`==`比較符號

In [49]:
p2, p3

(Point(2, 3), Point(0, 0))

In [50]:
p2 > p3

TypeError: '>' not supported between instances of 'Point' and 'Point'

#### 所以，該類別不支援比較運算子，如 `<`、`<=` 等。

#### 甚至 `==` 也無法按照預期運作 —— 它會使用記憶體位址來進行比較，而不是比較我們可能預期的 `x` 和 `y` 座標。


#### 對於 `<` 運算子，我們需要讓我們的類別實作 `__lt__` 方法，而對於 `==` 運算子，我們則需要實作 `__eq__` 方法。

#### 其他比較運算子可以透過實作不同的函式來支援，例如 `__le__`（`<=`）、`__gt__`（`>`）、`__ge__`（`>=`）等。

#### 接下來，我們將會為 `Point` 類別新增 `__lt__` 和 `__eq__` 方法。

#### 在這裡，我們會將一個 `Point` 物件視為比另一個更小，如果它更靠近原點（也就是具有較小的大小）。

In [52]:
del Point

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __abs__(self):
        return sqrt(self.x**2 + self.y**2)

    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return NotImplemented

    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented
    
    def __repr__(self):
        return f'{self.__class__.__name__}({self.x}, {self.y})'
        


In [53]:
p1, p2, p3 = Point(2, 3), Point(2, 3), Point(0, 0)

In [54]:
p1, p2, p1==p2

(Point(2, 3), Point(2, 3), True)

In [55]:
p2, p3, p2==p3

(Point(2, 3), Point(0, 0), False)

#### 所以，等號 `==` 已經可以用了

In [56]:
p4 = Point(1, 2)

In [57]:
abs(p1), abs(p4), p1<p4

(3.605551275463989, 2.23606797749979, False)

#### 不錯！基本上 `<` 和 `==` 都已經建好了。其他的呢？`<=` `>=` `>`

In [59]:
p1 > p4

True

#### 哦，既然我們已經實現了 `<` 和 `==`，這是否意味著 Python 神奇地實現了 `>` 運算子（也就是既不是 `<` 也不是 `==`）？

#### 不完全是，對了一半！實際上，由於 `p1` 和 `p4` 都是點，執行比較 `p1 > p4` 實際上和評估 `p4 < p1` 是一樣的——而 Python 確實自動為我們處理了這一點。

#### 但它並沒有實現其他比較運算子，比如 `>=` 和 `<=`。

In [60]:
p1 < p4

False

In [61]:
p1 <= p4

TypeError: '<=' not supported between instances of 'Point' and 'Point'

#### 現在，雖然我們可以以類似的方式進行，並使用相同的技巧來定義 `>=`、`<=` 和 `>`，但請注意，如果已經定義了 `<` 和 `==`，那麼：

> NOTE: iff = if and only if

> 
* `a <= b` iff `a < b or a == b`
> 
* `a > b` iff `not(a<b) and a != b`
> 
* `a >= b` iff `not(a<b)`

#### 因此，為了更通用，我們可以創建一個裝飾器，該裝飾器會實現這最後三個運算子，只要 `==` 和 `<` 已經被定義。我們接著可以裝飾**任何**只實作這兩個運算子的類別。

### Aaron注釋：
#### 這段程式碼的目的是，在確保類別 cls 已經實現了 __eq__ 和 __lt__ 方法的情況下，為這個類別自動添加 __le__（小於等於）、__gt__（大於）、__ge__（大於等於）這些比較方法。這樣，使用這些比較運算符時，就不需要手動實現它們。

#### dir() 是 Python 內建的一個函數，它返回一個列表，包含對象的所有屬性和方法名稱（以字符串形式）。

In [63]:
def complete_orderign(cls):
    if '__eq__' in dir(cls) and '__lt__' in dir(cls):
        cls.__le__ = lambda self, other: self < other or self == other
        cls.__gt__ = lambda self, other: not(self < other) and not (self == other)
        cls.__ge__ = lambda self, other: not(self < other)
    return cls

#### 實際上，上面的程式碼**並不是**一個好的實作方式。我們並沒有檢查類型是否相容，也沒有在適當的情況下返回 `NotImplemented`。我還使用了內聯運算子（`<` 和 `==`），而不是雙下劃線函數（`__lt__` 和 `__eq__`）。我只是為了簡單起見，因為稍後我們會使用更好的替代方案。

#### 例如，實現 `__ge__` 的更好的方式如下：

In [65]:
def ge_from_lt(self, other):
    # self >= other iff not(other < self)
    result = self.__lt__(other)
    if result is NotImplemented:
        return NotImplemented
    else:
        return not result


#### 你可能會想知道為什麼我使用 `__lt__` 而不是直接使用 `<` 運算子。這是因為我希望能夠實際查看操作的結果，而不會在操作未實現時拋出異常。我所實現的總排序裝飾器方式可能會導致無限迴圈，因為當我評估 `self < other` 時，如果拋出了異常，Python 會反射評估為 `other > self`，如果這也引發錯誤，Python 會嘗試再次反射這個操作，這樣就會進入無限迴圈（最終會以堆疊溢出結束）。這實際上是 Python 標準庫中 `complete_ordering` 裝飾器（稱為 `total_ordering`）的一個 bug，該 bug 在 3.4 版本中已經解決。

<font color=olive>這段文字提到的 complete_ordering（應該是指 Python 標準庫中的 @total_ordering 裝飾器）用來自動生成一組比較方法。total_ordering 裝飾器是 Python 內建的工具，可以根據你實現的某些比較方法（如 __eq__ 和 __lt__）自動生成其他比較方法（如 <=、>、>= 等）。但這段文字中提到了一些潛在的問題和一個 bug，在某些情況下，使用這些比較方法可能會導致無限迴圈。

@total_ordering 的工作原理
@total_ordering 裝飾器可以根據你定義的兩個或更多比較方法，自動生成其餘的比較方法。當你使用 @total_ordering 裝飾器時，你只需要實現其中兩個比較方法（如 __eq__ 和 __lt__），其他方法（如 __le__、__gt__、__ge__）會自動由裝飾器生成。

例如，假設你在 Point 類中定義了 __eq__ 和 __lt__，total_ordering 裝飾器會根據這兩個方法生成其他比較方法。



In [66]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __abs__(self):
        return sqrt(self.x**2 + self.y**2)
    
    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return NotImplemented

    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented
    
    def __repr__(self):
        return f'{self.__class__.__name__}({self.x}, {self.y})'

    

### <font color=olive> 原始碼分析
#### 關鍵部分：
@total_ordering 裝飾器：

這個裝飾器會基於你實現的 __eq__ 和 __lt__ 方法，自動生成 __le__、__gt__ 和 __ge__ 等其他比較方法。因此，你不需要手動寫出這些方法。
__eq__ 和 __lt__ 方法：

__eq__：檢查兩個 Point 是否相等，基於 x 和 y 坐標來比較。
__lt__：比較兩個 Point 的距離（即其原點到該點的距離），使用 abs(self) 和 abs(other) 來進行比較。
@total_ordering 生成的其他方法：

裝飾器會自動根據 __eq__ 和 __lt__ 生成 __le__（小於等於）、__gt__（大於）、__ge__（大於等於）等方法。
可能的問題與無限迴圈
在文字中提到，如果你在比較操作中使用了 == 和 <，但沒有正確處理 NotImplemented，可能會導致無限遞迴。這是因為當 Python 嘗試比較兩個對象（比如 self < other）時，如果這個操作未實現，Python 會反射地去嘗試 other > self，如果這也拋出錯誤，Python 又會回過頭來重新嘗試 self < other，這樣就會導致無限迴圈。

關於 bug
這個問題在 Python 3.4 中是一個已知的 bug，當時的 @total_ordering 裝飾器處理不當，會導致比較操作進入無限遞迴。這個 bug 已經在之後的版本中被修復。

總結
@total_ordering 裝飾器：這是 Python 的一個標準庫裝飾器，幫助你根據兩個比較方法（如 __eq__ 和 __lt__）自動生成其他比較方法。
在你的 Point 類中，使用 @total_ordering 裝飾器，Python 會自動生成 __le__、__gt__ 和 __ge__ 等方法。
若在 __lt__ 和 __eq__ 的比較中處理不當，可能會引發無限迴圈，這個問題在 Python 3.4 版本中是已知的 bug，後來被修復。

In [67]:
Point = complete_ordering(Point)

NameError: name 'complete_ordering' is not defined

In [68]:
Point = complete_ordering(Point)

NameError: name 'complete_ordering' is not defined

### Aaron 解決 NameError 的方法：
1. 定義 complete_ordering 裝飾器
如果你自己定義了 complete_ordering 裝飾器，請確保它正確地定義並可用。下面是一個簡單的 complete_ordering 裝飾器的範例：

```python
def complete_ordering(cls):
    if '__eq__' in dir(cls) and '__lt__' in dir(cls):
        cls.__le__ = lambda self, other: self < other or self == other
        cls.__gt__ = lambda self, other: not(self < other) and not (self == other)
        cls.__ge__ = lambda self, other: not(self < other)
    return cls
```
確保這個定義存在，並且在你調用 complete_ordering(Point) 之前已經執行過這段程式碼。

2. 使用 functools.total_ordering（如果你是想使用 Python 標準庫的裝飾器）
如果你是想使用 Python 標準庫中的 @total_ordering 裝飾器，可以將它導入並應用到你的 Point 類中：

```python

from functools import total_ordering

@total_ordering
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __abs__(self):
        return sqrt(self.x**2 + self.y**2)

    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        return NotImplemented

    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        return NotImplemented

    def __repr__(self):
        return '{0}({1},{2})'.format(self.__class__, self.x, self.y)
```

這樣就可以使用 @total_ordering 來自動生成比較方法了。

總結：
如果你使用的是自定義的 complete_ordering 裝飾器，請確保它已經定義並在使用前導入。
如果你想使用 Python 內建的 @total_ordering 裝飾器，請從 functools 模組導入它。

In [69]:
p1, p2, p3 = Point(1, 1), Point(3, 4), Point(3, 4)

In [71]:
abs(p1), abs(p2), abs(p3)

(1.4142135623730951, 5.0, 5.0)

In [72]:
p1 < p2, p1 <= p2, p1 > p2, p1 >= p2, p2 >= p3

TypeError: '<=' not supported between instances of 'Point' and 'Point'

#### 現在，`complete_ordering` 裝飾器也可以直接應用於任何定義了 `__eq__` 和 `__lt__` 的類別。

In [ ]:
@complete_ordering
class Grade:
    def __init__(self, score, max_score):
        self.score = score
        self.max_score = max_score
        self.score_percent = round(score / max_score * 100)

    def __repr__(self):
        return f'Grade({self.score}, {self.max_score})'
    
    def __eq__(self, other):
        if isinstance(other, Grade):
            return self.score_percent == other.score_percent
        else: 
            return NotImplement
        
    def __lt__(self, other):
        if isinstance(other, Grade):
            return self.score_percent < other.score_percent
        else:
            return NotImplemented

#### 通常，給定 `==` 操作符和僅 **一個** 其他比較操作符（`<`、`<=`、`>`、`>=`），其餘的比較操作符就可以被推導出來。

#### 我們的裝飾器強制要求使用 `==` 和 `<`，但我們可以讓它更靈活，強制要求使用 `==` 和其他任一比較操作符。這當然會讓我們的裝飾器變得更複雜，事實上，Python 在 `functools` 模組中已經內建了這樣的功能！

#### 它是一個叫做 `total_ordering` 的裝飾器。

#### 讓我們來看看它是如何運作的：

#### 還能這樣：

We have so far worked with decorating functions. This means we can decorate functions defined with a `def` statement (we can use the `@` syntax, or the long form). Since class methods are functions, they can be decorated too. Lambda expressions can also be decorated (using the long form).

But if you think about how our decorators work, they take a single parameter, a function, and return some other function - usually a closure that uses the original function that was passed as an argument.

We could use the same concept to accept, not a function, but a class instead. We could reference that class inside our decorator, modify it, and then return that modified class.

First we look at something called **monkey patching**. It boils down to modifying or extending our code at **run time**.

For example we can modify or add attributes to classes at run time. Modules too.

In Python, many of the classes we use can be modified at run time 
(built-ins like strings, lists, and so on, cannot).

But classes written in Python, such as the ones we write, and even library classes, as long as they are written in Python, not C, can. For example `Fraction` in the `fractions` module can be monkey patched.

Just because we can do something however, does not mean we should! Monkey patching can be extremely useful, but don't do it just because you can - as always there should be a real reason to do it, as we'll see in a bit.

Also, in general it is a bad idea to monkey patch the special methods `__???__` (such as `__len__`) as this will often not work due to how these methods are searched for by Python.

In [1]:
from fractions import Fraction

In [2]:
Fraction.speak = lambda self: 'This is a late parrot.'

In [3]:
f = Fraction(2, 3)

In [4]:
f

Fraction(2, 3)

In [5]:
f.speak()

'This is a late parrot.'

Yes, this is obviously nonsense, but you get the idea that you can add attributes to classes even if you do not have direct control over the class, or after your class has been defined.

If you want a more useful method, how about one that tells us if the Fraction is an integral number? (i.e. denominator is `1`)

In [6]:
Fraction.is_integral = lambda self: self.denominator == 1

In [7]:
f1 = Fraction(1, 2)
f2 = Fraction(10, 5)

In [8]:
f1.is_integral()

False

In [9]:
f2.is_integral()

True

Now, we can make this change to the class by calling a function to do it instead:

In [10]:
def dec_speak(cls):
    cls.speak = lambda self: 'This is a very late parrot.'
    return cls

In [11]:
Fraction = dec_speak(Fraction)

_(Hopefully the above code reminds you of decorators.)_

In [12]:
f = Fraction(10, 2)

In [13]:
f.speak()

'This is a very late parrot.'

We can use that function to decorate our custom classes too, using the short **@** syntax too.

In [14]:
@dec_speak
class Parrot:
    def __init__(self):
        self.state = 'late'

In [15]:
polly = Parrot()

In [16]:
polly.speak()

'This is a very late parrot.'

Using this technique we could for example add a useful *reciprocal* attribute to the Fraction class, but of course since it would probably be a one time kind of thing (how many Fraction classes are there that you will want to add a reciprocal to after all), there's no need for decorators. Decorators  are useful when they are able to be reused in more general ways.

In [17]:
Fraction.recip = lambda self: Fraction(self.denominator, self.numerator)

In [18]:
f = Fraction(2,3)

In [19]:
f

Fraction(2, 3)

In [20]:
f.recip()

Fraction(3, 2)

These example are quite trivial, and not very useful. 

So why bring this up? 

Because this same technique can be used for more interesting things.

As a first example, let's say you typically like to inspect various properties of an object for debugging purposes, maybe the memory address, it's current state (property values), and the time at which the debug info was generated.

In [21]:
from datetime import datetime, timezone

In [22]:
def debug_info(cls):
    def info(self):
        results = []
        results.append('time: {0}'.format(datetime.now(timezone.utc)))
        results.append('class: {0}'.format(self.__class__.__name__))
        results.append('id: {0}'.format(hex(id(self))))
        
        if vars(self):
            for k, v in vars(self).items():
                results.append('{0}: {1}'.format(k, v))
        
        # we have not covered lists, the extend method and generators,
        # but note that a more Pythonic way to do this would be:
        #if vars(self):
        #    results.extend('{0}: {1}'.format(k, v) 
        #                   for k, v in vars(self).items())
        
        return results
    
    cls.debug = info
    
    return cls

In [23]:
@debug_info
class Person:
    def __init__(self, name, birth_year):
        self.name = name
        self.birth_year = birth_year
        
    def say_hi():
        return 'Hello there!'

In [24]:
p1 = Person('John', 1939)

In [25]:
p1.debug()

['time: 2018-02-09 04:44:02.893951+00:00',
 'class: Person',
 'id: 0x2dfe29a4630',
 'name: John',
 'birth_year: 1939']

And of course we can decorate other classes this way too, not just a single class:

In [26]:
@debug_info
class Automobile:
    def __init__(self, make, model, year, top_speed_mph):
        self.make = make
        self.model = model
        self.year = year
        self.top_speed_mph = top_speed_mph
        self.current_speed = 0
    
    @property
    def speed(self):
        return self.current_speed
    
    @speed.setter
    def speed(self, new_speed):
        self.current_speed = new_speed

In [27]:
s = Automobile('Ford', 'Model T', 1908, 45)

In [28]:
s.debug()

['time: 2018-02-09 04:44:03.562898+00:00',
 'class: Automobile',
 'id: 0x2dfe29b3a58',
 'make: Ford',
 'model: Model T',
 'year: 1908',
 'top_speed_mph: 45',
 'current_speed: 0']

In [29]:
s.speed = 20

In [30]:
s.debug()

['time: 2018-02-09 04:44:03.898085+00:00',
 'class: Automobile',
 'id: 0x2dfe29b3a58',
 'make: Ford',
 'model: Model T',
 'year: 1908',
 'top_speed_mph: 45',
 'current_speed: 20']

In [31]:
from math import sqrt

In [32]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __abs__(self):
        return sqrt(self.x**2 + self.y**2)
    
    def __repr__(self):
        return 'Point({0},{1})'.format(self.x, self.y)

In [33]:
p1, p2, p3 = Point(2, 3), Point(2, 3), Point(0,0)

In [34]:
abs(p1)

3.605551275463989

In [35]:
p1, p2

(Point(2,3), Point(2,3))

In [36]:
p1 == p2

False

Hmm, we probably would have expected `p1` to be equal to `p2` since it has the same coordinates. But by default Python will compare memory addresses, since our class does not implement the `__eq__` method used for `==` comparisons.

In [37]:
p2, p3

(Point(2,3), Point(0,0))

In [38]:
p2 > p3

TypeError: '>' not supported between instances of 'Point' and 'Point'

So, that class does not support the comparison operators such as `<`, `<=`, etc. 

Even `==` does not work as expected - it will use the memory address instead of using a comparison of the `x` and `y` coordinates as we might probably expect.

For the `<` operator, we need our class to implement the `__lt__` method, and for `==` we need the `__eq__` method.

Other comparison operators are supported by implementing a variety of functions such as `__le__` (`<=`), `__gt__` (`>`), `__ge__` (`>=`).

We are going to add the `__lt__` and `__eq__` methods to our Point class.

We will consider a Point object to be smaller than another one if it is closer to the origin (i.e. smaller magnitude).

In [39]:
del Point

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __abs__(self):
        return sqrt(self.x**2 + self.y**2)
    
    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return NotImplemented
            
    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented
        
    def __repr__(self):
        return '{0}({1},{2})'.format(self.__class__.__name__, self.x, self.y)

In [40]:
p1, p2, p3 = Point(2, 3), Point(2, 3), Point(0,0)

In [41]:
p1, p2, p1==p2

(Point(2,3), Point(2,3), True)

In [42]:
p2, p3, p2==p3

(Point(2,3), Point(0,0), False)

As we can see, `==` now works as expected

In [43]:
p4 = Point(1, 2)

In [44]:
abs(p1), abs(p4), p1 < p4

(3.605551275463989, 2.23606797749979, False)

Great, so now we have `<` and `==` implemented. What about the rest of the operators: `<=`, `>`, `>=`?

In [45]:
p1 > p4

True

Ooh, since we have implemented `<` and `==`, does this mean Python magically implemented a `>` operator (i.e. not < and not ==)?

Not exactly! What happened is that since `p1` and `p4` are both points, running the comparison `p1 > p4` is really the same as evaluating `p4 < p1` - and Python did do that automatically for us.

But it has not implemented any of the others, such as `>=` and `<=`:

In [46]:
p1 <= p4

TypeError: '<=' not supported between instances of 'Point' and 'Point'

Now, although we could proceed in a similar way and define `>=`, `<=` and `>` using the same technique, observe that if `<` and `==` is defined then:

* `a <= b` iff `a < b or a == b`
* `a > b` iff `not(a<b) and a != b`
* `a >= b` iff `not(a<b)`

So, to be quite generic we could create a decorator that will implement these last three operators as long as `==` and `<` are defined. We could then decorate **any** class that implements just those two operators.

In [47]:
def complete_ordering(cls):
    if '__eq__' in dir(cls) and '__lt__' in dir(cls):
        cls.__le__ = lambda self, other: self < other or self == other
        cls.__gt__ = lambda self, other: not(self < other) and not (self == other)
        cls.__ge__ = lambda self, other: not (self < other)
    return cls

In reality, the code above is **NOT** a good implementation at all. We are not checking that the types are compatible and returning a `NotImplemented` result if appropriate. I am also using inline operators (`<` and `==`) instead of the dunder functions (`__lt__` and `__eq__`). I just kept it simple because we'll use a better alternative in a bit.

For example, a better way to implement `__ge__` would be as follows:

In [48]:
def ge_from_lt(self, other):
    # self >= other iff not(other < self)
    result = self.__lt__(other)
    if result is NotImplemented:
        return NotImplemented
    else:
        return not result

You may be wondering why I used `__lt__` instead of just using the `<` operator. This is because I want to actually look at the result of the operation without raising an exception if the operation is not implemented. The way I have the total ordering decorator implemented could cause an infinite loop because when I evaluate `self < other`, if an exception is raised, Python will reflect the evaluation to `other > self`, and if that raises an error as well, Python will try to reflect that operation too, and we get into an infinite loop (which eventually terminates in a stack overflow). This was actually a bug in Python's standard library implementation of a `complete_ordering` decorator (called `total_ordering`) that was resolved in 3.4.

In [49]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __abs__(self):
        return sqrt(self.x**2 + self.y**2)
    
    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return NotImplemented
            
    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented
        
    def __repr__(self):
        return '{0}({1},{2})'.format(self.__class__, self.x, self.y)

In [50]:
Point = complete_ordering(Point)        

In [51]:
p1, p2, p3 = Point(1, 1), Point(3, 4), Point(3, 4)

In [52]:
abs(p1), abs(p2), abs(p3)

(1.4142135623730951, 5.0, 5.0)

In [53]:
p1 < p2, p1 <= p2, p1 > p2, p1 >= p2, p2 > p2, p2 >= p3

(True, True, False, False, False, True)

Now the `complete_ordering` decorator can also be directly applied to any class that defines `__eq__` and `__lt__`.

In [54]:
@complete_ordering
class Grade:
    def __init__(self, score, max_score):
        self.score = score
        self.max_score = max_score
        self.score_percent = round(score / max_score * 100)
     
    def __repr__(self):
        return 'Grade({0}, {1})'.format(self.score, self.max_score)
    
    def __eq__(self, other):
        if isinstance(other, Grade):
            return self.score_percent == other.score_percent
        else:
            return NotImplemented
    
    def __lt__(self, other):
        if isinstance(other, Grade):
            return self.score_percent < other.score_percent
        else:
            return NotImplemented
        

In [55]:
g1 = Grade(10, 100)
g2 = Grade(20, 30)
g3 = Grade(5, 50)

In [56]:
g1 <= g2, g1 == g3, g2 > g3

(True, True, True)

Often, given the `==` operator and just **one** of the other comparison operators (`<`, `<=`, `>`, `>=`), then all the rest can be derived.

Our decorator insisted on `==` and `<`. but we could make it better by insisting on `==` and any one of the other operators. This will of course make our decorator more complicated, and in fact, Python has this precise functionality built in to the, you guessed it, `functools` module!

It is a decorator called `total_ordering`. 

Let's see it in action:

In [57]:
from functools import total_ordering

In [58]:
@total_ordering
class Grade:
    def __init__(self, score, max_score):
        self.score = score
        self.max_score = max_score
        self.score_percent = round(score / max_score * 100)
     
    def __repr__(self):
        return 'Grade({0}, {1})'.format(self.score, self.max_score)
    
    def __eq__(self, other):
        if isinstance(other, Grade):
            return self.score_percent == other.score_percent
        else:
            return NotImplemented
    
    def __lt__(self, other):
        if isinstance(other, Grade):
            return self.score_percent < other.score_percent
        else:
            return NotImplemented

In [59]:
g1, g2 = Grade(80, 100), Grade(60, 100)

In [60]:
g1 >= g2, g1 > g2

(True, True)

Or we could also do it this way:

In [61]:
@total_ordering
class Grade:
    def __init__(self, score, max_score):
        self.score = score
        self.max_score = max_score
        self.score_percent = round(score / max_score * 100)
     
    def __repr__(self):
        return 'Grade({0}, {1})'.format(self.score, self.max_score)
    
    def __eq__(self, other):
        if isinstance(other, Grade):
            return self.score_percent == other.score_percent
        else:
            return NotImplemented
    
    def __gt__(self, other):
        if isinstance(other, Grade):
            return self.score_percent > other.score_percent
        else:
            return NotImplemented

In [62]:
g1, g2 = Grade(80, 100), Grade(60, 100)

In [63]:
g1 >= g2, g1 > g2, g1 <= g2, g1 < g2

(True, True, False, False)